In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.embeddings import CacheBackedEmbeddings
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda

llm = ChatOpenAI(temperature=0.1)

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

loader = UnstructuredFileLoader("./files/chapter_one.docx")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cache_dir = LocalFileStore("./.cache")

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
     embeddings,
     cache_dir,
)

vectorstore = Chroma.from_documents(docs, cached_embeddings)

retriever = vectorstore.as_retriever()


map_doc_prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        """
        Use the following portion of a long document to see if any of the text is relevant to anser the question
        Return any relevant text verbatim.
        ------
        {context}
        """
    ),
    ("human", "{question}")
])

map_doc_chain = map_doc_prompt | llm

def map_docs(inputs):
    documents = inputs['documents']
    question = inputs['question']
    results = []
    for document in documents:
        result = map_doc_chain.invoke({
            "context": document.page_content,
            "question": question
        }).content
        results.append(result)
    results = "\n\n".join(results)
    return results

map_chain = {"documents": retriever, "question": RunnablePassthrough()} | RunnableLambda(map_docs)


final_prompt = ChatPromptTemplate.from_messages([
    (
        "system", 
        """
        Given the following extracted parts of a long docement and a question, create a final anwer.
        If you don't know the answer, just say that you don't know.
        Don't try to make up an answer.
        ------
        {context}
        """
    ),
    ("human", "{question}")
])

chain = {"context": map_chain, "question": RunnablePassthrough()} | final_prompt | llm

chain.invoke("What is the name of Oldwyn’s mechanical doll?")

AIMessage(content="The name of Oldwyn's mechanical doll is Vesper.")